In [1]:
import pandas as pd

df = pd.read_csv('germany_city_data.csv')

df.head()

,population_rank,city_id,city_name,longitude,latitude,elevation,state_id,image_url
0,c001,Berlin,Berlin,13.4050,52.5200,NaN,Berlin,https://raw.githubusercontent.com/Prasadmadhus...
1,c002,Hamburg,Hamburg,9.9937,53.5511,NaN,Hamburg,https://raw.githubusercontent.com/Prasadmadhus...
2,c003,Munich,Munich,11.5820,48.1351,NaN,Bavaria,https://raw.githubusercontent.com/Prasadmadhus...
3,c004,Colonge,Colonge,6.9603,50.9375,NaN,North_Rhine-Westphalia,https://raw.githubusercontent.com/Prasadmadhus...
4,c005,Frankfurt_am_Main,Frankfurt am Main,8.6821,50.1109,NaN,Hesse,https://raw.githubusercontent.com/Prasadmadhus...


In [2]:
print(df.columns)

Index(['population_rank', 'city_id', 'city_name', 'longitude', 'latitude',
       'elevation', 'state_id', 'image_url'],
      dtype='object')


In [4]:
import requests
import time

def get_elevation(latitude, longitude):
    url = f"https://api.open-elevation.com/api/v1/lookup?locations={latitude},{longitude}"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        elevation = response.json()["results"][0]["elevation"]
        return elevation
    except Exception as e:
        print(f"Error for ({latitude}, {longitude}): {e}")
        return None

elevations = []

for lat, lon in zip(df["latitude"], df["longitude"]):
    elevations.append(get_elevation(lat, lon))
    time.sleep(0.1)  # be polite to the free API

df["elevation"] = elevations

In [6]:
df = df.drop(columns=['population_rank'])

df.head()

,city_id,city_name,longitude,latitude,elevation,state_id,image_url
0,Berlin,Berlin,13.4050,52.5200,45.0,Berlin,https://raw.githubusercontent.com/Prasadmadhus...
1,Hamburg,Hamburg,9.9937,53.5511,11.0,Hamburg,https://raw.githubusercontent.com/Prasadmadhus...
2,Munich,Munich,11.5820,48.1351,518.0,Bavaria,https://raw.githubusercontent.com/Prasadmadhus...
3,Colonge,Colonge,6.9603,50.9375,56.0,North_Rhine-Westphalia,https://raw.githubusercontent.com/Prasadmadhus...
4,Frankfurt_am_Main,Frankfurt am Main,8.6821,50.1109,104.0,Hesse,https://raw.githubusercontent.com/Prasadmadhus...


In [7]:
import geopandas as gpd
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(df["longitude"], df["latitude"])]

gdf = gpd.GeoDataFrame(
    df,
    geometry=geometry,
    crs="EPSG:4326"  # WGS84 (GPS)
)

In [8]:
output_path = "germany_cities.geojson"

gdf.to_file(
    output_path,
    driver="GeoJSON",
    encoding="utf-8"
)